# Scenario 2 - Randomized counts for Tableau
### For the Digital Health Research Infrastructure Specialist Position

**Jessa Green  |  jessa@jessagreen.com  |  (920) 850-4950  |  [My LinkedIn Profile](https://www.linkedin.com/in/jessagreen/)**

Below is the process I followed to create a fictional dataset based on numbers I was able to gather from your public information. This dataset was used to populate the tables presented in the Tableau dashboard provided as the solution to Scenario 2 in the interview process.

> Note: These are not real data or numbers for The PRIDE Study. Everything shown here is a fictional scenario based a few real-world numbers I gathered.

In [1]:
# Import needed libraries/packages
import pandas as pd
import numpy as np
import random
from datetime import datetime
import calendar

In [2]:
# Create variables based on publicly known PRIDE Studies numbers

# Participant mid-year counts from medical journal
ps_mid18 = 10952
ps_mid19 = 13731
# Fictional participant counts created
ps_mid20 = 16436
ps_mid21 = 19853
# Participant count shown in my PS account dashboard on 2022-01-21
ps22 = 23952


# The PRIDE Study response percentage for the 2017 AQ
aq17 = 0.658
# The PRIDE Study response percentage for the 2018 AQ (11 month window)
aq18pt = 0.479
# Create a 12-month percentage based on a basic average
aq18 = (aq18pt / 11) * 12
print("This gives us a 'full year' response rate of {} to use for 2018 AQ".format(round(aq18, 3)))

This gives us a 'full year' response rate of 0.523 to use for 2018 AQ


In [3]:
# Create fictional response rates for each year post 2018

aq19 = np.random.uniform((aq18-0.12), (aq17+0.12))
aq20 = np.random.uniform((aq18-0.12), (aq17+0.12))
aq21 = np.random.uniform((aq18-0.12), (aq17+0.12))

print("This gives us response rates of: \n2019 AQ: {:.3f} \n2020 AQ: {:.3f} \n2021 AQ: {:.3f}".format(aq19, aq20, aq21))


This gives us response rates of: 
2019 AQ: 0.406 
2020 AQ: 0.453 
2021 AQ: 0.765


### Create function to generate a set of monthly counts
The below function will generate a set of 12 random counts that will total a given amount when summed.

In [4]:
# Now create fictional monthly response counts based on annual numbers

def gen_monthly_counts(n, total):
# Return a randomly chosen list of n positive integers summing to total.
    dividers = sorted(random.sample(range(1, total), n - 1))
    return [a - b for a, b in zip(dividers + [total], [0] + dividers)]


In [5]:
# Create dictionary of years and counts to create our dataset for Tableau
ps_dict = {2018: ps_mid18, 2019: ps_mid19, 2020: ps_mid20, 2021: ps_mid21}


### Create the dataset contents
Here we generate the entire lists consisting of each of the years, the months, and the participation counts using the monthly randomizer function so that our annual counts match our summed monthly counts for each year.

In [6]:
month_counts = []
month_years = []
just_years = []

for key,value in ps_dict.items():
    month_counts.extend(gen_monthly_counts(12, value))
    for i in range(12):      
        new_date = datetime.strptime((str(i+1) + '/' + str(key)),'%m/%Y').date()
        month_years.append(new_date)
        just_years.append(key)


In [7]:
df = pd.DataFrame(list(zip(just_years, month_years, month_counts)))
df.columns =['Yr', 'Date', 'Participants']

In [8]:
df.head(12)

,Yr,Date,Participants
0,2018,2018-01-01,120
1,2018,2018-02-01,3146
2,2018,2018-03-01,50
3,2018,2018-04-01,323
4,2018,2018-05-01,2237
5,2018,2018-06-01,437
6,2018,2018-07-01,472
7,2018,2018-08-01,210
8,2018,2018-09-01,291
9,2018,2018-10-01,312


### Randomized monthly counts fit annual counts

The below comparison confitms that my randomized dataset has created random monthly counts for participation to be 
used in our visualized data:

In [9]:
print(df.query('Yr == 2018').Participants.sum(), '<-- Sum of randomized monthly counts for 2018')
print(ps_dict[2018], '<-- Count from the created annual variables at the top')

10952 <-- Sum of randomized monthly counts for 2018
10952 <-- Count from the created annual variables at the top


In [10]:
print(df.query('Yr == 2021').Participants.sum(), '<-- Sum of randomized monthly counts for 2021')
print(ps_dict[2021], '<-- Count from the created annual variables at the top')

19853 <-- Sum of randomized monthly counts for 2021
19853 <-- Count from the created annual variables at the top


In [11]:
df.drop('Yr', axis=1, inplace=True)
df.head()

,Date,Participants
0,2018-01-01,120
1,2018-02-01,3146
2,2018-03-01,50
3,2018-04-01,323
4,2018-05-01,2237


### Convert Random Counts to Individual Responses
Finally, I take the monthly counts and assign them each individual "responses" by state to align with the number of responses each month/year randomized above.

In [12]:
test = dict(zip(df.Date, df.Participants))
state_list = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
gid_list = ['Agender', 'Cisgender Man', 'Cisgender Woman', 'Genderqueer', 
       'Man', 'Non-Binary', 'Questioning', 'Transgender Man', 'Transgender Woman',
       'Two-spirit', 'Woman', 'Another gender identity']
so_list = ['Asexual', 'Bisexual', 'Gay', 'Lesbian', 'Pansexual', 'Queer',
               'Questioning', 'Same-gender loving', 'Straight/Heterosexual', 'Two-spirit', 
               'Another sexual orientation']

In [13]:
def age_range(age):
    if age < 24:
        age_group = '18-23'
    elif age < 30:
        age_group = '24-29'
    elif age < 40:
        age_group = '30-39'
    elif age < 50:
        age_group = '40-49'
    elif age < 60:
        age_group = '50-59'
    elif age < 70:
        age_group = '60-69'
    elif age < 80:
        age_group = '70-79'        
    return age_group

print(age_range(43))


40-49


In [14]:
def get_percent(year):
    global percent
    if year == 2017:
        percent = aq17
    elif year == 2018:
        percent = aq18
    elif year == 2019:
        percent = aq19
    elif year == 2020:
        percent = aq20
    elif year == 2021:
        percent = aq21
    return percent

print(get_percent(2020))

0.4528129937919227


In [24]:
#all_part_states_list = []
#all_part_dates_list = []
#all_part_ages_list = []

df_list = []
counter = 1

for i,j in test.items():
    for count in range(j):
        participant_id = str(counter)
        date_joined = i
        state = random.choices(state_list)
        age = random.choice(range(18,78))
        age_group = age_range(age)
        curr_gid = random.choices(gid_list)
        curr_so = random.choices(so_list)
        yr_percent = get_percent(i)
        active_status = np.random.choice(2, p=[(1-yr_percent),yr_percent])
        counter += 1
        
        df_list.append({'date_joined': date_joined,
                        'participant_id': participant_id,
                        'state': state[0],
                        'age': age,
                        'age_group': age_group,
                        'curr_gid': curr_gid[0],
                        'curr_so': curr_so[0],
                        'active': active_status})
        
df_full = pd.DataFrame(df_list, columns = ['participant_id', 'date_joined', 'state', 'age', 'age_group',
                        'curr_gid', 'curr_so', 'active'])

In [25]:
df_full.head() 

,participant_id,date_joined,state,age,age_group,curr_gid,curr_so,active
0,1,2018-01-01,TX,71,70-79,Cisgender Woman,Lesbian,1
1,2,2018-01-01,NE,64,60-69,Genderqueer,Another sexual orientation,1
2,3,2018-01-01,VA,39,30-39,Agender,Gay,0
3,4,2018-01-01,FL,45,40-49,Genderqueer,Two-spirit,1
4,5,2018-01-01,OR,18,18-23,Genderqueer,Asexual,1


In [26]:
df_full.shape

(60972, 8)

### Export the dataset to CSV to use in Tableau

In [27]:
df_full.to_csv('pride_study_participation_new.csv', index=False)


*Known participant numbers gathered from Journal of the American Medical Informatics Association, 2019, Vol. 26, No.8-9, pg.745-746